# 04: Sandbox Code Execution

This notebook showcases the `sandbox_mcp_server`, which provides a secure and isolated environment for executing arbitrary code. This allows the agent to perform dynamic calculations, data manipulation, and other tasks that can be solved with code, without compromising the host system.

## Getting Started

Before running this notebook, ensure that the Docker services for Chapter 06 are up and running. You can start them by navigating to the `chapter-06-advanced-multi-agent-orchestration` directory and executing the lifecycle script:

```bash
./start-chapter-resources.sh
```

Also, ensure you have an SSH session with port forwarding for OpenWebUI (`8902`), the Agent Gateway (`8083`), and the Sandbox MCP server (`8082`):

```bash
ssh -L 8902:localhost:8902 -L 8083:localhost:8083 -L 8082:localhost:8082 user@remote-server
```

## 1. Executing Python Code

The `ExecuteCode` tool allows you to run Python code within the sandbox. The sandbox is isolated and has no network access, making it safe for executing untrusted code. The last line of the code should be an expression that yields the result.


In [2]:
import openai
import json

# Configuration
AGENT_GATEWAY_URL = "http://localhost:8083/v1" # Agent Gateway for chat interactions
MODEL_ID = "agentic-framework/scientific-agent-v1"
API_KEY = "not-a-real-key" # Dummy API key if auth is disabled

client = openai.OpenAI(
    base_url=AGENT_GATEWAY_URL,
    api_key=API_KEY,
)

python_code = """
import math
result = math.sqrt(16) * 10
result
"""

user_query = f"""Execute the following Python code in the sandbox: ```python
{python_code}
```"""

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("--- Agent Response ---")
    if response.choices:
        print(response.choices[0].message.content)
    else:
        print("The agent did not return any choices.")
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Sending query to agent: Execute the following Python code in the sandbox: ```python

import math
result = math.sqrt(16) * 10
result

```
Failed to connect to the Agent Gateway: Connection error.


## 2. Executing Shell Commands

You can also execute shell commands in the sandbox. This is useful for basic system interactions or running command-line utilities.


In [3]:
import openai
import json

# Configuration (same as above)
AGENT_GATEWAY_URL = "http://localhost:8083/v1"
MODEL_ID = "agentic-framework/scientific-agent-v1"
API_KEY = "not-a-real-key"

client = openai.OpenAI(
    base_url=AGENT_GATEWAY_URL,
    api_key=API_KEY,
)

shell_code = """
ls -la /app
"""

user_query = f"""Execute the following shell command in the sandbox: ```shell
{shell_code}
```"""

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("--- Agent Response ---")
    if response.choices:
        print(response.choices[0].message.content)
    else:
        print("The agent did not return any choices.")
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Sending query to agent: Execute the following shell command in the sandbox: ```shell

ls -la /app

```
Failed to connect to the Agent Gateway: Connection error.


## 3. Executing JavaScript Code

The sandbox also supports JavaScript execution. This can be useful for web-related tasks or data processing in a JavaScript environment.


In [ ]:
import openai
import json

# Configuration (same as above)
AGENT_GATEWAY_URL = "http://localhost:8083/v1"
MODEL_ID = "agentic-framework/scientific-agent-v1"
API_KEY = "not-a-real-key"

client = openai.OpenAI(
    base_url=AGENT_GATEWAY_URL,
    api_key=API_KEY,
)

js_code = """
const a = 5;
const b = 7;
console.log(a * b);
"""

user_query = f"Execute the following JavaScript code in the sandbox: ```javascript
{js_code}
```"

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("
--- Agent Response ---")
    if response.choices:
        print(response.choices[0].message.content)
    else:
        print("The agent did not return any choices.")
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


## 4. Generating Plots with Python

The `ExecuteCode` tool can also capture and return plots generated by Python libraries like `matplotlib` or `seaborn`. To enable this, set the `generate_plot` parameter to `True`. Remember to explicitly create figure objects (e.g., `fig, ax = plt.subplots()`) and avoid `plt.show()`. The figure object should be the last expression in your code block for it to be captured.

**Important Note for Plot Rendering:**
For plot rendering to work correctly in your local Docker Compose environment, ensure you have set the `SANDBOX_MCP_SERVER_PUBLIC_URL` environment variable in your `.env` file (e.g., `SANDBOX_MCP_SERVER_PUBLIC_URL=http://localhost:8082`). This variable tells the sandbox server the public URL it should use when generating plot links.


In [ ]:
import openai
import json

# Configuration (same as above)
AGENT_GATEWAY_URL = "http://localhost:8083/v1"
MODEL_ID = "agentic-framework/scientific-agent-v1"
API_KEY = "not-a-real-key"

client = openai.OpenAI(
    base_url=AGENT_GATEWAY_URL,
    api_key=API_KEY,
)

python_plot_code = """
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 2*np.pi, 400)
y = np.sin(x**2)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title('A simple sine wave')
fig
"""

user_query = f"Execute the following Python code in the sandbox and generate a plot: ```python
{python_plot_code}
```"

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("
--- Agent Response ---")
    if response.choices:
        print(response.choices[0].message.content)
    else:
        print("The agent did not return any choices.")
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


This notebook demonstrated how to execute code in the sandbox and generate plots. In the next notebook, we will explore advanced multi-agent orchestration.
